<a href="https://colab.research.google.com/github/sougatshome/4D-pocket/blob/master/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Tue Jul 27 19:25:33 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
'205 Abstract - Line chart 1.gsheet'
'209: CONTROL SYSTEMS.gdoc'
'Among.Us.v2020.9.9s.zip (Unzipped Files)'
'ARBIE REC'
 Assgn-2-EC208-191EC152.gdoc
 Bhaswati_Shome_51584.pdf
 books
 Classroom
'Colab Notebooks'
 Day2_p17.ipynb
 Documents
'EC 280 Project report.docx'
'hamsa dhwani.mp3'
'Honors request form ECE 2019-23.gdoc'
'Honors request form ECE 2019-23.odt'
'How to get started with Drive.pdf'
'image captioning'
 impulse#16
'lagi (1).mp4'
'lagi lagana.mp4'
'lagi L.mp4'
 lagi.mp4
 LearnerLicense.pdf
 me.jpeg
 ml
'My Drive'
'nmt-awfw-bxh - Jun 12, 2021.gjam'
'rahul ID.jpeg'
 Report.gdoc
 Resume.gdoc
'scs-amjc-dqa - Jan 28, 2021.gjam'
'shalaka ID.jpeg'
 Sougat
'sougat ID.jpg'
"Sougat's Resume - new.pdf"
 Sougat_s_Resume.pdf
'To be Checked.gdoc'
'Untitled document (1).gdoc'
'Untitled document.gdoc'
 vest_oldweights
 yolov3
'Your big idea.gslides'
'ywf-xirc-uh

**1) Clone the Darknet**



In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15287, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 15287 (delta 0), reused 1 (delta 0), pack-reused 15283
Receiving objects: 100% (15287/15287), 13.69 MiB | 22.54 MiB/s, done.
Resolving deltas: 100% (10377/10377), done.


**2) Compile Darknet using Nvidia GPU**


In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo "Koala" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/obj

In [ ]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-07-27 19:26:50--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  42.9MB/s    in 4.0s    

2021-07-27 19:26:54 (38.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/worker (139).jpg  
  inflating: data/obj/worker (139).txt  
  inflating: data/obj/worker (143).jpg  
  inflating: data/obj/worker (143).txt  
  inflating: data/obj/worker (165).jpg  
  inflating: data/obj/worker (165).txt  
  inflating: data/obj/worker (178).jpg  
  inflating: data/obj/worker (178).txt  
  inflating: data/obj/worker (179).jpg  
  inflating: data/obj/worker (186).jpg  
  inflating: data/obj/worker (186).txt  
  inflating: data/obj/worker (187).jpg  
  inflating: data/obj/worker (187).txt  
  inflating: data/obj/worker (9).txt  
  inflating: data/obj/worker (77).txt  
  inflating: data/obj/worker (83).txt  
  inflating: data/obj/0.jpg          
  inflating: data/obj/0.txt          
  inflating: data/obj/2.jpg          
  inflating: data/obj/2.txt          
  inflating: data/obj/16.jpg         
  inflating: data/obj/16.txt         
  inflating: data/obj/18.jpg         
  inflating: data/obj/18.txt         
  infla

In [ ]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

15 0.368359 0.829167 0.457031 0.333333

['15', '0.368359', '0.829167', '0.457031', '0.333333']
0 data/obj/pos_1597.txt
0 0.368359 0.829167 0.457031 0.333333
15 0.493000 0.497000 0.658000 0.874000

['15', '0.493000', '0.497000', '0.658000', '0.874000']
1 data/obj/pos_685.txt
0 0.493000 0.497000 0.658000 0.874000
15 0.359766 0.668750 0.380469 0.659722

['15', '0.359766', '0.668750', '0.380469', '0.659722']
2 data/obj/pos_2009.txt
0 0.359766 0.668750 0.380469 0.659722
15 0.539844 0.416667 0.160938 0.272222

['15', '0.539844', '0.416667', '0.160938', '0.272222']
2 data/obj/pos_2009.txt
0 0.539844 0.416667 0.160938 0.272222
15 0.526000 0.850667 0.224000 0.298667

['15', '0.526000', '0.850667', '0.224000', '0.298667']
3 data/obj/843.txt
0 0.526000 0.850667 0.224000 0.298667
15 0.490159 0.540885 0.914714 0.783196

['15', '0.490159', '0.540885', '0.914714', '0.783196']
4 data/obj/pos_626.txt
0 0.490159 0.540885 0.914714 0.783196
15 0.498000 0.496000 0.636000 0.962000

['15', '0.498000', '0.496

In [ ]:
import glob
images_list = glob.glob("data/obj/*.jpg")
print(images_list)

['data/obj/pos_686.jpg', 'data/obj/worker (75).jpg', 'data/obj/pos_639.jpg', 'data/obj/pos_655.jpg', 'data/obj/pos_2145.jpg', 'data/obj/pos_1595.jpg', 'data/obj/pos_492.jpg', 'data/obj/654.jpg', 'data/obj/pos_1505.jpg', 'data/obj/pos_629.jpg', 'data/obj/pos_2016.jpg', 'data/obj/596.jpg', 'data/obj/pos_894.jpg', 'data/obj/worker (178).jpg', 'data/obj/worker (83).jpg', 'data/obj/pos_1957.jpg', 'data/obj/797.jpg', 'data/obj/pos_1359.jpg', 'data/obj/worker (123).jpg', 'data/obj/pos_677.jpg', 'data/obj/823.jpg', 'data/obj/pos_2142.jpg', 'data/obj/pos_34.jpg', 'data/obj/843.jpg', 'data/obj/pos_305.jpg', 'data/obj/pos_1939.jpg', 'data/obj/pos_385.jpg', 'data/obj/pos_2011.jpg', 'data/obj/148.jpg', 'data/obj/pos_438.jpg', 'data/obj/708.jpg', 'data/obj/pos_679.jpg', 'data/obj/pos_547.jpg', 'data/obj/pos_2055.jpg', 'data/obj/pos_1654.jpg', 'data/obj/66.jpg', 'data/obj/pos_2060.jpg', 'data/obj/pos_1364.jpg', 'data/obj/pos_1967.jpg', 'data/obj/pos_2121.jpg', 'data/obj/pos_1812.jpg', 'data/obj/pos_2

In [ ]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
 total_bbox = 113162, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.750198), count: 2, class_loss = 0.139718, iou_loss = 0.049156, total_loss = 0.188874 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.890315), count: 3, class_loss = 0.026080, iou_loss = 0.060455, total_loss = 0.086535 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 113167, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.826652), count: 2, class_loss = 0.000175, iou_loss = 0.021110, total_loss = 0.021286 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.827040), count: 3, class_loss = 0.015959, iou_loss = 0.069361, total_loss = 0.085320 
v3 (ms